In [8]:
import pandas as pd

In [2]:
users=pd.read_csv('users_20200506.csv')

In [3]:
users.head()

,createdAt,updatedAt,userId,wwid,firstName,lastName,region,country,sector,functionRole,position,sysRole,tableauBookmarksOrder,gauserid
0,1583838294925,1583838294925,JCharle5,152868417,Jo,Charles,EU,GB,CONS,NFCF,PM,USER,[],ecda0944f01c7ae205b260ae68ddea1e6d6e5173a94f06...
1,1587545438215,1587545438215,aschmid,29520330,Andreas,Schmid,EU,CH,CONS,BUF,PM,USER,[],5ecb44e213e1b37175ef516f57399009f1a649862efe71...
2,1564128185789,1576292583525,bhoppe1,149002015,Bastian Steffen,Hoppe,EU,DE,CONS,NFCF,PM,USER,[145],57a0d0643dfc70e2f154b5e0f058eff7d1976ee940bcb4...
3,1573145569364,1576292588045,JPERES,19022896,Jean Michel,PERES,EU,FR,CONS,NFCF,PM,USER,[],f896ddc4839d56789a6bf16891a914ffa0aa3e6a0bd6e8...
4,1570562570985,1576292587492,jduggan9,368424,JEFFRY,DUGGAN,NaN,US,MD,COE,PA,USER,[],2e17ed9916f46322d1acf2fd259595cd9b4e6e9841eb18...


In [4]:
 date_mapping_source=pd.read_csv('Date Mapping.csv')

In [5]:
date_mapping_source.head()

,Date,Month Date
0,1-Jan-18,1
1,2-Jan-18,2
2,3-Jan-18,3
3,4-Jan-18,4
4,5-Jan-18,5


In [9]:
behaviour_source=pd.read_csv('User_Behavior_Data_20200506.csv')

In [10]:
behaviour_source.head()

,createdAt,workbookid,userId,wwid,firstName,lastName,region,country,sector,functionRole,`position`
0,6/17/2019 8:01,ba53e3d6-3bb2-4a20-950a-2089d182374a,Yqin150,152049173,Yanjun,Qin,WW,CN,MD,COE,PA
1,6/17/2019 8:40,0e9531cb-3958-4b29-98cb-8a71981465ff,Yqin150,152049173,Yanjun,Qin,WW,CN,MD,COE,PA
2,6/18/2019 3:06,32e116aa-16c0-4e54-9443-ef27cf66d117,Yqin150,152049173,Yanjun,Qin,WW,CN,MD,COE,PA
3,6/18/2019 3:06,efe22284-003b-4136-b08a-a4806cd8b303,Yqin150,152049173,Yanjun,Qin,WW,CN,MD,COE,PA
4,6/18/2019 3:08,32e116aa-16c0-4e54-9443-ef27cf66d117,Yqin150,152049173,Yanjun,Qin,WW,CN,MD,COE,PA


In [13]:
def source_loader(database_config='', user_source='', behavior_source='', date_mapping_source='' ,source_refine_list=[]):

    """Load input csv sources, generate necessary columns and return a raw data frame combined the csv sources.

    Params: 
        database_config(string): a json file access to database,when given, user_source and behavior_source are no needed.
        user_source(string): user_source file path.
        behavior_source(string): behavior_source file path.
        date_mapping_source (string): date_mapping_source file path.
    Returns:
        dataframe: 3 Dataframes which combined the input sources : mba_source, user_source, date_mapping_source.
    """
    # from where to get data
    if len(database_config)==0:
        # load csv files
        user_source = pd.read_csv(user_source)
        behavior_source = pd.read_csv(behavior_source)
    else:
        # access to databases via configuration
        f=open(database_config,encoding='utf-8')
        config=json.load(f)
        con = pymysql.connect(host=config['host'],port=config['port'],user=config['user'],passwd=config['passwd'],db=config['database'],charset='utf8')

        sql_query_user=config['sql_query_user']
        sql_query_behavior=config['sql_query_behavior']

        # get data
        user_source=pd.read_sql(sql_query_user,con)
        behavior_source = pd.read_sql(sql_query_behavior, con)

        # close connection
        con.close()

    # load date mapping
    date_mapping_source = pd.read_csv(date_mapping_source)

    # Only Active user is used.
    #user_source = user_source[user_source['status'] == 'ACTIVE']

    # handle user_source value 'NA', as user from US, region is 'NA', not EMPTY!
    user_source['region'] = user_source['region'].astype(str)
    for index, row in user_source.iterrows():
        if row['region'] == 'nan': 
            user_source.at[index, 'region'] = 'NA'

    # drop none value.
    user_source.dropna(subset=['region','sector'],inplace=True)

    # handle behavior_source value 'NA', as user from US, region is 'NA', not EMPTY!
    behavior_source['region'] = behavior_source['region'].astype(str)
    for index, row in behavior_source.iterrows():
        if row['region'] == 'nan':
            behavior_source.at[index, 'region'] = 'NA'

    # filter those workbook ids' length don't equal to 36 
    behavior_source = behavior_source[behavior_source['workbookid'].apply(lambda x: len(x)==36)]
    behavior_source = behavior_source[behavior_source['workbookid'].apply(lambda x: set('?=@#$%&') & set(x)==set())]

    # generate mba_source from behavior_source
    behavior_source['Year Month Day'] = pd.to_datetime(behavior_source['createdAt']).dt.strftime('%Y%m%d')

    # Aggregate workbook id based on wwid and year month day
    MBA_groups = behavior_source.groupby(['wwid', 'Year Month Day'])['workbookid'].apply(lambda x: list(set(x)))
    mba_source = MBA_groups.reset_index()
    mba_source.columns=['wwid', 'Year Month Day', 'Workbook IDs']

    # concatenate user and mba_source dataset
    mba_source = mba_source.merge(user_source, left_on='wwid', right_on='wwid')

    # Read data mapping csv and format time
    date_mapping_source['formated_time'] = pd.to_datetime(date_mapping_source['Date']).dt.strftime('%Y%m%d')
    date_mapping_source.set_index('formated_time', inplace=True)

    # Extract month and date from mba_source period column
    mba_source['period'] = mba_source['Year Month Day'].apply(lambda x: date_mapping_source.loc[str(x)]['Month Date'])

    # transform period to string
    mba_source['period']=mba_source['period'].apply(lambda x:str(int(x)))

    # refine the needed columns to analysis
    mba_source=mba_source[source_refine_list]

    return user_source, mba_source, date_mapping_source

In [14]:
# load sources
source_refine_list = ['Workbook IDs', 'region', 'sector', 'functionRole', 'position', 'period']
user_source='users_20200506.csv'
behavior_source='User_Behavior_Data_20200506.csv'
date_mapping_source='Date Mapping.csv'
db_config=''
user_table, source_table, date_table = source_loader(database_config=db_config,user_source=user_source,behavior_source=behavior_source,
                                           date_mapping_source=date_mapping_source, source_refine_list=source_refine_list)

In [15]:
user_table.head()

,createdAt,updatedAt,userId,wwid,firstName,lastName,region,country,sector,functionRole,position,sysRole,tableauBookmarksOrder,gauserid
0,1583838294925,1583838294925,JCharle5,152868417,Jo,Charles,EU,GB,CONS,NFCF,PM,USER,[],ecda0944f01c7ae205b260ae68ddea1e6d6e5173a94f06...
1,1587545438215,1587545438215,aschmid,29520330,Andreas,Schmid,EU,CH,CONS,BUF,PM,USER,[],5ecb44e213e1b37175ef516f57399009f1a649862efe71...
2,1564128185789,1576292583525,bhoppe1,149002015,Bastian Steffen,Hoppe,EU,DE,CONS,NFCF,PM,USER,[145],57a0d0643dfc70e2f154b5e0f058eff7d1976ee940bcb4...
3,1573145569364,1576292588045,JPERES,19022896,Jean Michel,PERES,EU,FR,CONS,NFCF,PM,USER,[],f896ddc4839d56789a6bf16891a914ffa0aa3e6a0bd6e8...
4,1570562570985,1576292587492,jduggan9,368424,JEFFRY,DUGGAN,NA,US,MD,COE,PA,USER,[],2e17ed9916f46322d1acf2fd259595cd9b4e6e9841eb18...


In [16]:
source_table.head()

,Workbook IDs,region,sector,functionRole,position,period
0,[391f15d6-6036-463e-8596-8eded82c1c79],EU,CONS,BUF,PD,4
1,[391f15d6-6036-463e-8596-8eded82c1c79],EU,CONS,BUF,PD,1
2,[391f15d6-6036-463e-8596-8eded82c1c79],EU,CONS,BUF,PD,2
3,"[40164491-b87e-43ec-abd3-18323d5939b8, 72104bd...",NA,CHQ,COE,PD,2
4,"[72104bde-df71-4aca-bac4-4f42220cdce9, 3fc85d5...",NA,CHQ,COE,PD,6


In [29]:
workbook_click=behaviour_source.groupby('wwid')['workbookid'].value_counts()
#workbook_clicks = workbook_click.reset_index()

In [30]:
workbook_click.to_csv('group.csv')

In [26]:
workbook_click=behaviour_source.groupby('wwid')['workbookid']

In [33]:
behaviour_source.groupby('wwid')['workbookid'].value_counts().unstack().fillna(0)

workbookid,/tableau/workbook?id=1,/tableau/workbook?id=39,03215b46-2912-49b3-8ffe-ea7ca3655f5b,059282d8-af15-43df-aa63-11150433c1a5,09906980-b557-43e2-b2b4-90f7c84ed5b7,09f6abda-b8a3-4a7f-9a83-91838ac450a3,0bd2e4db-0dcf-4690-9e77-1a0c9b844739,0c56badf-8679-452f-a6bd-5471350ff30e,0da33704-2c79-493a-812a-93445a9d1bb9,0dd1ba04-8a3d-4e97-9877-430303b92ac3,...,fc0aa5af-ba45-4a33-839a-e7c37a55d46b,fcf78bd7-302c-4f66-be48-c5817f5d68cb,fdbe275e-f125-4072-88a0-37cf11d20d76,fdeec40f-fe1b-4a30-a95a-f6d4189c1110,fe1a9fbd-9b8b-41af-8591-b747f3592fef,fe4b421-5100-4849-ab3d-80dc0d3a3861],ff32c2e2-aab1-49d8-9b66-b6241b47ca74,ffd38253-1cef-4ce0-a191-7d2484bbde4c,kbook?id=391f15d6-6036-463e-85https:,leau/workbook?id=1c281add-2ff0-4d10-
wwid,,,,,,,,,,,,,,,,,,,,,
36761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702315010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
702316396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
702316649,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
users_filtered = behaviour_source[(behaviour_source.region == 'EU') & (behaviour_source.sector == 'CONS')]

In [12]:
users_filtered.head()

,createdAt,workbookid,userId,wwid,firstName,lastName,region,country,sector,functionRole,`position`
33,6/19/2019 6:59,7fe4b421-5100-4849-ab3d-80dc0d3a3861,DPhilli6,31001577,David,Phillips,EU,GB,CONS,COE,PM
34,6/20/2019 7:52,0da33704-2c79-493a-812a-93445a9d1bb9,gfeng18,152812438,Feng,Gao,EU,CN,CONS,BUF,PA
35,6/20/2019 7:53,e9e0ef50-6785-46e1-a462-a3a52408f919,gfeng18,152812438,Feng,Gao,EU,CN,CONS,BUF,PA
36,6/20/2019 12:14,391f15d6-6036-463e-8596-8eded82c1c79,DPhilli6,31001577,David,Phillips,EU,GB,CONS,COE,PM
37,6/20/2019 12:15,1dc35060-51f7-4364-aad7-33a20d0deeb2,cjone107,1040374,Corey,Jones,EU,GB,CONS,COE,PA


In [13]:
users_filtered=users_filtered.reset_index()
del users_filtered['index']

In [14]:
users_filtered.head()

,createdAt,workbookid,userId,wwid,firstName,lastName,region,country,sector,functionRole,`position`
0,6/19/2019 6:59,7fe4b421-5100-4849-ab3d-80dc0d3a3861,DPhilli6,31001577,David,Phillips,EU,GB,CONS,COE,PM
1,6/20/2019 7:52,0da33704-2c79-493a-812a-93445a9d1bb9,gfeng18,152812438,Feng,Gao,EU,CN,CONS,BUF,PA
2,6/20/2019 7:53,e9e0ef50-6785-46e1-a462-a3a52408f919,gfeng18,152812438,Feng,Gao,EU,CN,CONS,BUF,PA
3,6/20/2019 12:14,391f15d6-6036-463e-8596-8eded82c1c79,DPhilli6,31001577,David,Phillips,EU,GB,CONS,COE,PM
4,6/20/2019 12:15,1dc35060-51f7-4364-aad7-33a20d0deeb2,cjone107,1040374,Corey,Jones,EU,GB,CONS,COE,PA


In [15]:
workbook_click=users_filtered.groupby('wwid')['workbookid'].value_counts()

In [16]:
workbook_click

wwid       workbookid                          
36761      391f15d6-6036-463e-8596-8eded82c1c79    4
39997      391f15d6-6036-463e-8596-8eded82c1c79    3
           459bee2f-f6d6-42fe-b2d1-1267ea2cdc94    2
           6fcfd350-6240-4279-a799-30812f8718bf    1
           e4beb8da-0225-456b-a9cf-9bd94a45b31c    1
                                                  ..
702316649  f07001d7-8c24-43d8-acb9-83f460a33f86    1
702317970  3145f28f-dc44-45bd-9b33-7728b6994ecb    2
           391f15d6-6036-463e-8596-8eded82c1c79    2
           1a262cce-a032-4319-bb06-4dadf0988567    1
           7fe4b421-5100-4849-ab3d-80dc0d3a3861    1
Name: workbookid, Length: 5158, dtype: int64

In [17]:
workbook_click.to_csv('clicks.csv')

In [43]:
clicks=pd.read_csv('clicks.csv')

In [45]:
del clicks['Unnamed: 0']
clicks.head()

,wwid,workbook_id,clicks
0,36761,13.0,4
1,39997,13.0,3
2,39997,14.0,2
3,39997,227.0,1
4,39997,20.0,1


In [46]:
clicks.to_csv('clicks.csv')

In [40]:
clicks.dropna(subset = ["workbook_id"], inplace=True)

In [50]:
workbook_click.unstack().fillna(0)

workbookid,/tableau/workbook?id=39,059282d8-af15-43df-aa63-11150433c1a5,09906980-b557-43e2-b2b4-90f7c84ed5b7,09f6abda-b8a3-4a7f-9a83-91838ac450a3,0bd2e4db-0dcf-4690-9e77-1a0c9b844739,0c56badf-8679-452f-a6bd-5471350ff30e,0da33704-2c79-493a-812a-93445a9d1bb9,0dd1ba04-8a3d-4e97-9877-430303b92ac3,0e9531cb-3958-4b29-98cb-8a71981465ff,14c145d3-36e0-4389-8a67-f59e47975f80,...,fbb404d2-a280-4019-87d5-e487d90caffd,fc0aa5af-ba45-4a33-839a-e7c37a55d46b,fcf78bd7-302c-4f66-be48-c5817f5d68cb,fdeec40f-fe1b-4a30-a95a-f6d4189c1110,fe1a9fbd-9b8b-41af-8591-b747f3592fef,fe4b421-5100-4849-ab3d-80dc0d3a3861],ff32c2e2-aab1-49d8-9b66-b6241b47ca74,ffd38253-1cef-4ce0-a191-7d2484bbde4c,kbook?id=391f15d6-6036-463e-85https:,leau/workbook?id=1c281add-2ff0-4d10-
wwid,,,,,,,,,,,,,,,,,,,,,
36761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702108277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
702161048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
702164602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Method 1: Alternating Least Squares

The numerical values of implicit feedback describe the frequency of actions e.g. how many times a J&J Employee clicks on a particular Report. We leverage these implicit numbers given by the users to capture the confidence of user’s preferences or no preferences. The preference (pui) of user ‘u’ to item ‘i’ is captured by the following equation:

<img src="equation1.PNG">

where ‘rui’ is the recorded value of the item (e.g. number of times you click on a report). So, if the recorded value is greater than 0, the preference is 1 and if it is equal to 0, the preference is 0.
Now, let’s say if you just listen to a song only once, it might not mean you liked it, it could have been on auto-play mode in your YouTube. Similarly, if the recorded value is zero, does not mean you don’t like the song, it might mean you never knew that song even existed or you haven’t discovered it yet. So, in order to capture this relation of preferences, we need to specify the notion of confidence:

<img src="Capture.PNG">

<br>In general, as ‘rui’ grows, we have a stronger indication that the user indeed likes the item. ‘cui’, measures our confidence in observing ‘pui’. We can notice, even for a user item pair with 0 ‘rui’ meaning no preference with the item, we have an associated minimal confidence of 1. And this confidence for ‘pui’ equal to 1, increases as the evidence for positive preference increases.</br>


The rate of increase is controlled by the constant tuning parameter α.
Preferences are assumed to be the inner products: pui = (Transpose)U. X

We need to account for the varying confidence levels
Optimization should account for all possible user-item pairs, rather than only those corresponding to observed data
Embedding/factors are computed by minimizing the following cost function:

<img src="cost.PNG">

In [1]:
import findspark
findspark.init()

In [2]:
import os
import math
import datetime
import pyspark
import pyspark.sql.functions as sf
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
from pyspark import SparkConf, SparkContext
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
from pyspark import SparkContext, SparkConf, SQLContext
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.ml.recommendation import ALS
# from pyspark.sql import Row
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql.types import TimestampType
from pyspark.sql import SparkSession

In [3]:
conf = SparkConf().setAppName('RepReco').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [48]:
# load job Clicks file into rdd
clicks_raw_data = sc.textFile('clicks.csv')
clicks_raw_data_header = clicks_raw_data.take(1)[0]
clicks_data = clicks_raw_data.filter(lambda line: line != clicks_raw_data_header).map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),int(tokens[2]))).cache()

print ("train size is ", clicks_data.count())
print('\nColumns are:', clicks_raw_data_header)
clicks_data.take(3)

train size is  4864

Columns are: wwid,workbook_id,clicks


[(36761, 13, 4), (39997, 13, 3), (39997, 14, 2)]

In [49]:
rddTraining, rddValidating, rddTesting = clicks_data.randomSplit([6,2,2], seed=1001)

#Add click data in the training model
nbValidating = rddValidating.count()
nbTesting    = rddTesting.count()

print("Training: %d, validation: %d, test: %d" % (rddTraining.count(), nbValidating, rddTesting.count()))

Training: 2954, validation: 945, test: 965


In [50]:
def howFarAreWe(model, against, sizeAgainst):
    againstNoRatings = against.map(lambda x: (int(x[0]), int(x[1])) )
    againstWiRatings = against.map(lambda x: ((int(x[0]),int(x[1])), int(x[2])) )
    predictions = model.predictAll(againstNoRatings).map(lambda p: ( (p[0],p[1]), p[2]) )
    predictionsAndRatings = predictions.join(againstWiRatings).values()    
    return sqrt(predictionsAndRatings.map(lambda s: (s[0] - s[1]) ** 2).reduce(add) / float(sizeAgainst))

In [35]:
rddTraining.take(3)

[(36761, '391f15d6-6036-463e-8596-8eded82c1c79', 4),
 (39997, '6fcfd350-6240-4279-a799-30812f8718bf', 1),
 (164648, '3145f28f-dc44-45bd-9b33-7728b6994ecb', 1)]

In [51]:
model = ALS.trainImplicit(rddTraining, 5, 5, float(0.1),alpha=float(10))

In [52]:
#finding best set of parameters
ranks  = [5,10,15,20]
reguls = [0.1, 1,10]
iters  = [5,10,20]
alpha = [10, 20, 40]

finalModel = None
finalRank  = 0
finalRegul = float(0)
finalIter  = -1
finalDist   = float(300)
finalAlpha = float(0)

#[START train_model]
for cRank, cRegul, cIter, cAlpha in itertools.product(ranks, reguls, iters, alpha):
    model = ALS.trainImplicit(rddTraining, cRank, cIter, float(cRegul),alpha=float(cAlpha))
    dist = howFarAreWe(model, rddValidating, nbValidating)
    if dist < finalDist:
        print(cIter, cRank,cAlpha,cRegul)
        print("Best so far:%f" % dist)
        finalModel = model
        finalRank  = cRank
        finalRegul = cRegul
        finalIter  = cIter
        finalDist  = dist
        finalAlpha  = cAlpha 
#[END train_model]

print("Rank %i" % finalRank) 
print("Regul %f" % finalRegul) 
print("Iter %i" % finalIter)  
print("Dist %f" % finalDist) 
print("Alpha %f" % finalAlpha)

5 5 10 0.1
Best so far:30.978897
5 5 40 0.1
Best so far:30.977418
10 5 10 0.1
Best so far:30.967678
10 5 20 0.1
Best so far:30.965265
10 5 40 0.1
Best so far:30.952411
20 5 40 0.1
Best so far:30.945180
10 5 40 10
Best so far:30.944385
20 5 40 10
Best so far:30.941331
10 15 40 10
Best so far:30.940799
Rank 15
Regul 10.000000
Iter 10
Dist 30.940799
Alpha 40.000000


In [53]:
model = ALS.trainImplicit(rddTraining, rank=finalRank, iterations=finalIter, lambda_= float(finalRegul),alpha=float(finalAlpha))
# Calculate all predictions
rddTesting_withoutclicks = rddTesting.map(lambda r: ((r[0], r[1])))
predictions = model.predictAll(rddTesting_withoutclicks).map(lambda r: ((r[0], r[1]), (r[2])))
predictions.take(3)
# user id, workbook_id, actual clickss,pred clickss -> df below
rates_and_preds = rddTesting.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions) 
rates_and_preds.take(3)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 34.14991882263968


In [54]:
rates_and_preds.take(3)

[((39997, 13), (3.0, -0.01565532542306422)),
 ((164648, 20), (1.0, 0.7502400116466508)),
 ((173145, 15), (1.0, 0.7204138324836867))]

In [55]:
def get_counts_and_averages(ID_and_ratings_tuple):    
    nratings = len(ID_and_ratings_tuple[1]) 
    return ID_and_ratings_tuple[0], (nratings, sum([float(val) for val in ID_and_ratings_tuple[1]])/nratings)

In [58]:
report_ID_with_ratings_RDD = (clicks_data.map(lambda x: (x[1], x[2])).groupByKey())
report_ID_with_ratings_RDD_updated = report_ID_with_ratings_RDD.map(lambda x : (x[0], list(x[1])))
report_ID_with_avg_ratings_RDD = report_ID_with_ratings_RDD_updated.map(get_counts_and_averages)  # count and average rating
report_rating_counts_RDD = report_ID_with_avg_ratings_RDD.map(lambda x: (int(x[0]), x[1][0]))    # rating count per job
report_rating_counts_RDD.cache()
report_rating_counts_RDD.take(3)

[(13, 823), (14, 158), (227, 9)]

In [59]:
report_ID_with_avg_ratings_RDD.take(3)

[(13, (823, 19.997569866342648)),
 (14, (158, 3.3797468354430378)),
 (227, (9, 1.6666666666666667))]

In [61]:
# load reports category file into rdd

reports_raw_data = sc.textFile("reports.csv")
reports_raw_data_header = reports_raw_data.take(1)[0]

reports_data = reports_raw_data.filter(lambda line: line!=reports_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1])).cache()
print('\nColumns are:', reports_raw_data_header)
reports_data.take(3)


Columns are: numerical_id,id


[(1, 'e12f7746-6dc1-42a5-9c96-93451dff608f'),
 (2, '7a1e6d11-3f9f-43de-8fa7-20b13bf3fc2a'),
 (3, 'ba53e3d6-3bb2-4a20-950a-2089d182374a')]

In [69]:
# get user-wise reports clicked
all_users_ratings_RDD = clicks_data.map(lambda x: (x[0], x[1])).groupByKey()
all_users_ratings_RDD = all_users_ratings_RDD.map(lambda x : (x[0], list(x[1])))    # reports clicked by each user

### finding unrated reports by each user- we will use this set for model's prediction/recommendations
report_ids = set(reports_data.map(lambda x : x[0]).toLocalIterator()) # list of all report ids
unrated_reports_RDD = all_users_ratings_RDD.map(lambda x: (x[0], list((report_ids) - set(x[1]))))

# #create user_id and unrated report id pairs
unrated_userreports_RDD = unrated_reports_RDD.flatMap(lambda x : [(x[0],i) for i in x[1]])


In [65]:
all_users_ratings_RDD.take(10)

[(36761, [13]),
 (39997, [13, 14, 227, 20]),
 (101849, [13]),
 (103380, [10, 13]),
 (164648, [13, 83, 11, 15, 16, 20]),
 (173145, [20, 10, 12, 13, 15]),
 (200068, [13, 15, 12, 21, 20, 10, 17, 11]),
 (363106, [13, 15, 16]),
 (377575, [13]),
 (406898, [12, 13, 17, 14, 20, 10, 11, 15])]

In [71]:
unrated_userreports_RDD.take(5)

[(36761, 1), (36761, 2), (36761, 3), (36761, 4), (36761, 5)]

In [74]:
# # #model predictions for each user and not clicked report pairs
recommendations_RDD = model.predictAll(unrated_userreports_RDD)
recommended_reports_rating_RDD = recommendations_RDD.map(lambda x: (x.product,(x.user, x.rating)))
recommended_reports_rating_RDD.cache()
print (recommended_reports_rating_RDD.take(10))

[(315, (52000470, 0.3828855265189147)), (386, (52000470, -0.005842662358757245)), (324, (52000470, -0.009733771027338314)), (408, (52000470, -0.007484627375054967)), (11, (52000470, 0.7033322421786886)), (14, (52000470, 0.5810631878438727)), (24, (52000470, 0.03187909938085373)), (302, (52000470, 0.13228422910947027)), (146, (52000470, -0.023427440394520216)), (195, (52000470, 0.04628480598891266))]


In [73]:
recommendations_RDD.take(10)

[Rating(user=52000470, product=315, rating=0.3828855265189147),
 Rating(user=52000470, product=386, rating=-0.005842662358757245),
 Rating(user=52000470, product=324, rating=-0.009733771027338314),
 Rating(user=52000470, product=408, rating=-0.007484627375054967),
 Rating(user=52000470, product=11, rating=0.7033322421786886),
 Rating(user=52000470, product=14, rating=0.5810631878438727),
 Rating(user=52000470, product=24, rating=0.03187909938085373),
 Rating(user=52000470, product=302, rating=0.13228422910947027),
 Rating(user=52000470, product=146, rating=-0.023427440394520216),
 Rating(user=52000470, product=195, rating=0.04628480598891266)]

In [76]:
# #     # converting id into int for report_clicks_count RDD to perform join

# join workbook_id with numeric workbook id, predicted rating for report and total number of ratings received by each report
recommendations_rating_title_and_count_RDD = recommended_reports_rating_RDD.join(reports_data).join(report_rating_counts_RDD)
recommendations_rating_title_and_count_RDD = recommendations_rating_title_and_count_RDD.map(lambda r: (r[0], r[1][0][1], r[1][0][0][0],round(r[1][0][0][1],2),r[1][1]))

In [77]:
recommendations_rating_title_and_count_RDD.take(5)

[(324, 'bfc19bb7-3053-4494-86f7-258db9a2ca66', 52000470, -0.01, 1),
 (324, 'bfc19bb7-3053-4494-86f7-258db9a2ca66', 152832913, -0.02, 1),
 (324, 'bfc19bb7-3053-4494-86f7-258db9a2ca66', 30050304, -0.01, 1),
 (324, 'bfc19bb7-3053-4494-86f7-258db9a2ca66', 42002455, -0.01, 1),
 (324, 'bfc19bb7-3053-4494-86f7-258db9a2ca66', 152818537, -0.0, 1)]

In [78]:
recommendations_rating_title_and_count_RDD = recommendations_rating_title_and_count_RDD.map(lambda x: (x[2],(x[0],x[1], x[3],x[4])))
recommendations_rating_title_and_count_RDD.take(3)

[(52000470, (324, 'bfc19bb7-3053-4494-86f7-258db9a2ca66', -0.01, 1)),
 (152832913, (324, 'bfc19bb7-3053-4494-86f7-258db9a2ca66', -0.02, 1)),
 (30050304, (324, 'bfc19bb7-3053-4494-86f7-258db9a2ca66', -0.01, 1))]

In [80]:
# filter only those reports which have been clicked by atleast 10 users
# take only top5 reports by sorting based on preference confidence
top_reports = recommendations_rating_title_and_count_RDD.groupBy(lambda x : x[0])\
                               .map(lambda x : list(x[1]))\
                               .map(lambda r: [i for i in r if i[1][3] > 10])\
                               .map(lambda a: [i for i in sorted(a, key=lambda x: -x[1][2])[:5]])   

#preparing dataframe to insert in Database
rec_reports_df = top_reports.map(lambda x: [(i[0],i[1][0],i[1][1],i[1][2]) for i in x]).flatMap(lambda x: x).toDF()\
                                .withColumnRenamed("_1", "user_id")\
                                .withColumnRenamed("_2", 'report_recommendations')\
                                .withColumnRenamed("_3", 'Workbook_id')\
                                .withColumnRenamed("_4", 'preference_confidence')\
                                .withColumnRenamed("_5", "total_clicks")
                
# #final recommendation engine dataframe to be saved in Database
#final_df_rec_eng = rec_reports_df.withColumn("rec_date", sf.lit(datetime.datetime.now()).cast(TimestampType()))   
final_df_rec_eng = final_df_rec_eng.withColumn("rec_number", sf.row_number().over(Window.partitionBy("user_id").orderBy(desc("preference_confidence"))))    
final_df_rec_eng.show(15)

+---------+----------------------+--------------------+---------------------+--------------------+----------+
|  user_id|report_recommendations|         Workbook_id|preference_confidence|            rec_date|rec_number|
+---------+----------------------+--------------------+---------------------+--------------------+----------+
| 10024312|                    20|e4beb8da-0225-456...|                 0.93|2020-07-06 11:17:...|         1|
| 10024312|                    16|83455e0f-e24e-4e3...|                 0.86|2020-07-06 11:17:...|         2|
| 10024312|                    14|459bee2f-f6d6-42f...|                 0.78|2020-07-06 11:17:...|         3|
| 10024312|                    22|1dc35060-51f7-436...|                 0.67|2020-07-06 11:17:...|         4|
| 10024312|                   315|b95c4206-1600-453...|                 0.64|2020-07-06 11:17:...|         5|
| 34010215|                    20|e4beb8da-0225-456...|                 0.62|2020-07-06 11:17:...|         1|
| 34010215